# Cognitive Modeling - Assignment 2
Yorick Juffer \
s1993623

## Libraries

In [396]:
import math
import random
from model import Model
from dmchunk import Chunk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Functions

In [397]:
t_0 = 0.011
a = 1.1
b = 0.15
add_noise = True

def noise(s):
    rand = random.uniform(0.001, 0.999)
    return s * math.log((1 - rand)/rand)

def time_to_pulses(time, t_0 = t_0, a = a, b = b, add_noise = add_noise):  
    pulses = 0
    pulse_duration = t_0
    
    while time >= pulse_duration:
        time = time - pulse_duration
        pulses = pulses + 1
        pulse_duration = a * pulse_duration + add_noise * noise(b * a * pulse_duration)
        
    return pulses

def pulses_to_time(pulses, t_0 = t_0, a = a, b = b, add_noise = add_noise):
    time = 0
    pulse_duration = t_0
    
    while pulses > 0:
        time = time + pulse_duration
        pulses = pulses - 1
        pulse_duration = a * pulse_duration + add_noise * noise(b * a * pulse_duration)
    
    return time

## Experiment

In [398]:
def experiment(n, t):

    fore_period = [0.4, 0.8, 1.2, 1.6]

    for i in range(n): 
        
        participant_model = Model()

        # train
        fore_period_rng = random.choice(fore_period)
        pulses = time_to_pulses(fore_period_rng)
        chunk = Chunk(name = "time" + str(i), slots = {"type": "time", "value": pulses})
        participant_model.add_encounter(chunk)
        participant_model.time += 1

        pastFP = fore_period_rng

        # trials
        for j in range(t):

            reaction_time = 0.39
            fore_period_rng = random.choice(fore_period)

            # pre stimulus
            blend_pattern = Chunk(name = "fore_period", slots = {"type": "time"})
            temp = participant_model.retrieve_blended_trace(blend_pattern, "value")

            difference = fore_period_rng - pulses_to_time(temp[0])
            participant_model.time += temp[1]
            
            if(difference >= 0.05):
                reaction_time =  reaction_time - 0.05
            elif(difference <= 0):
                reaction_time =  reaction_time
            else: 
                reaction_time =  reaction_time - difference

            participant = i
            currentFP = fore_period_rng

            if(i == 0):
                results = np.array([[participant, reaction_time, currentFP, pastFP]])
            else:
                results = np.append(results, [[participant, reaction_time, currentFP, pastFP]], axis=0)

            pastFP = fore_period_rng
            
            # post stimulus
            pulses = time_to_pulses(fore_period_rng)
            chunk = Chunk(name = "time" + str(i) + str(j), slots = {"type": "time", "value": pulses})
            participant_model.add_encounter(chunk)
            participant_model.time += 1
            
            print("Participant: {}".format(i), end="\r")

    return results

# runs the experiment and generates data 896
data = experiment(25, 1000)
dataFrame = pd.DataFrame(data)
dataFrame.columns = ["pp", "RT", "FP", "FPp"]


## Data

           RT                              
FPp       0.4       0.8       1.2       1.6
FP                                         
0.4  0.373338  0.379951  0.381420  0.382963
0.8  0.361268  0.370165  0.373270  0.374382
1.2  0.355526  0.363917  0.367234  0.370196
1.6  0.351845  0.360114  0.363243  0.367375


In [399]:
df = dataFrame
df.drop('pp', inplace=True, axis=1)
grouped_df = df.groupby(["FP", "FPp"]).mean().unstack()
print(grouped_df)
plt.plot(grouped_df, linestyle='--', marker='o', label = ["0.4", "0.8", "1.2", "1.6"])
plt.legend(loc='upper right')
plt.show()
